In [1]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm
import re
import collections
import itertools
from statistics import mean

In [2]:
remove_marks_regex = re.compile("[\,\(\)\[\]\*:;¿¡]|<.*?>")
shift_marks_regex = re.compile("([?!\.])")

unk = 0
sos = 1
eos = 2

In [3]:
def normalize(text):
    text = text.lower()
    text = remove_marks_regex.sub("", text)
    text = shift_marks_regex.sub(r" \1", text)
    return text

def parse_line(line):
    line = normalize(line.strip())
    # src - target 각각의 토큰을 리스트화
    src, trg = line.split("\t")
    src_tokens = src.strip().split()
    trg_tokens = trg.strip().split()
    return src_tokens, trg_tokens

def build_vocab(tokens):
    # 모든 무장에서 토큰 등장 횟수 확인
    counts = collections.Counter(tokens)
    sorted_counts = sorted(counts.items(), key=lambda c: c[1], reverse=True)
    word_list = ["<UNK>",  "<SOS>", "<EOS>"] + [x[0] for x in sorted_counts]
    word_dict = dict((w, i) for i, w in enumerate(word_list))
    return word_list, word_dict

def words2tensor(words, word_dict, max_len, padding=0):
    # 종료 태그
    words = words + ["<EOS>"]
    words = [word_dict.get(w, 0) for w in words]
    seq_len = len(words)
    if seq_len < max_len + 1:
        words = words + [padding] * (max_len + 1 - seq_len)
    return torch.tensor(words, dtype=torch.int64), seq_len

In [4]:
class TranslationPairDataset(Dataset):
    def __init__(self, path, max_len=15):
        def filter_pair(p):
            # 단어수가 많은 문장 제거
            return not (len(p[0]) > max_len or len(p[1]) > max_len)
        with open(path, encoding='utf8') as fp:
            pairs = map(parse_line, fp)
            pairs = filter(filter_pair, pairs)
            pairs = list(pairs)
        src = [p[0] for p in pairs]
        trg = [p[1] for p in pairs]
        self.src_word_list, self.src_word_dict = build_vocab(itertools.chain.from_iterable(src))
        self.trg_word_list, self.trg_word_dict = build_vocab(itertools.chain.from_iterable(trg))
        self.src_data = [words2tensor(words, self.src_word_dict, max_len) for words in src]
        self.trg_data= [words2tensor(words, self.trg_word_dict, max_len, -100) for words in trg]
        
    def __len__(self):
        return len(self.src_data)
    
    def __getitem__(self, idx):
        src, lsrc = self.src_data[idx]
        trg, ltrg = self.trg_data[idx]
        return src, lsrc, trg, ltrg

In [5]:
batch_size = 64
max_len = 10
path = "d:/dataset/spa-eng/spa.txt"
ds = TranslationPairDataset(path, max_len=max_len)
loader = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=0)

In [6]:
class Encoder(nn.Module):
    def __init__(self, num_embeddings, embedding_dim=50, hidden_size=50, num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
    
    def forward(self, x, h0=None, l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, l, batch_first=True)
            _, h = self.lstm(x, h0)
        return h

class Decoder(nn.Module):
    def __init__(self, num_embeddings, embedding_dim=50, hidden_size=50, num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, num_embeddings)
    
    def forward(self, x, h, l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, l, batch_first=True)
        x, h = self.lstm(x, h)
        if l is not None:
            x = nn.utils.rnn.pad_packed_sequence(x, batch_first=True, padding_value=0)[0]
        x = self.linear(x)
        return x, h

In [7]:
def translate(input_str, enc, dec, max_len=15, device="cpu"):
    # 입력 문자열을 수치화해서 Tensor로 변환
    words = normalize(input_str).split()
    input_tensor, seq_len = words2tensor(words, 
        ds.src_word_dict, max_len=max_len)
    input_tensor = input_tensor.unsqueeze(0)
    # 엔코더에서 사용하므로 입력값의 길이도 리스트로 만들어둔다
    seq_len = [seq_len]
    # 시작 토큰 준비
    sos_inputs = torch.tensor(sos, dtype=torch.int64)
    input_tensor = input_tensor.to(device)
    sos_inputs = sos_inputs.to(device)
    # 입력 문자열을 엔코더에 넣어서 컨텍스트 얻기
    ctx = enc(input_tensor, l=seq_len)
    # 시작 토큰과 컨텍스트를 디코더의 초깃값으로 설정
    z = sos_inputs
    h = ctx
    results = []
    for i in range(max_len):
        # Decoder로 다음 단어 예측
        o, h = dec(z.view(1, 1), h)
        # 선형 계층의 출력이 가장 큰 위치가 다음 단어의 ID
        wi = o.detach().view(-1).max(0)[1]
        if wi.item() == eos:
            break
        results.append(wi.item())
        # 다음 입력값으로 현재 출력 ID를 사용
        z = wi
    # 기록해둔 출력 ID를 문자열로 변환
    return " ".join(ds.trg_word_list[i] for i in results)

In [8]:
enc = Encoder(len(ds.src_word_list), 100, 100, 2)
dec = Decoder(len(ds.trg_word_list), 100, 100, 2)
translate("I am a student.", enc, dec)
enc.to("cuda:0")
dec.to("cuda:0")
opt_enc = optim.Adam(enc.parameters(), 0.002)
opt_dec = optim.Adam(dec.parameters(), 0.002)
loss_f = nn.CrossEntropyLoss()

In [11]:
def to2D(x):
    shapes = x.shape
    return x.reshape(shapes[0] * shapes[1], -1)

for epoch in range(30):
    enc.train(), dec.train()
    losses = []
    for x, lx, y, ly  in tqdm.tqdm(loader):
        # x packed sequence를 위해 소스 길이로 내림차순 정렬
        lx, sort_idx = lx.sort(descending=True)
        x, y, ly = x[sort_idx], y[sort_idx], ly[sort_idx]
        x, y = x.to("cuda:0"), y.to("cuda:0")
        ctx = enc(x, l=lx)
        
        ly, sort_idx = ly.sort(descending=True)
        y = y[sort_idx]
        h0 = (ctx[0][:, sort_idx, :], ctx[1][:, sort_idx, :])
        z = y[:, :-1].detach()
        z[z==-100] = 0
        o, _ = dec(z, h0, l=ly-1)
        loss = loss_f(to2D(o[:]), to2D(y[:, 1:max(ly)]).squeeze())
        enc.zero_grad(), dec.zero_grad()
        loss.backward()
        opt_enc.step(), opt_dec.step()
        losses.append(loss.item())
    enc.eval(), dec.eval()
    print(epoch, mean(losses))
    
    with torch.no_grad():
        print(translate("I am a student.", enc, dec, max_len=max_len, device="cuda:0"))
        print(translate("He likes to eat pizza.", enc, dec, max_len=max_len, device="cuda:0"))
        print(translate("She is my mother.", enc, dec, max_len=max_len, device="cuda:0"))

100%|██████████████████████████████████████████████████████████████████████████████| 1661/1661 [00:54<00:00, 30.49it/s]


0 5.457710967732797
un poco .
a tom .
a mi casa .


100%|██████████████████████████████████████████████████████████████████████████████| 1661/1661 [00:54<00:00, 30.51it/s]


1 3.48005776703896
un estudiante .
a tom que se va a la cena .
a mi madre .


100%|██████████████████████████████████████████████████████████████████████████████| 1661/1661 [00:54<00:00, 30.60it/s]


...

27 0.4808443382445191
un estudiante .
a comer pizza .
mi madre .


100%|██████████████████████████████████████████████████████████████████████████████| 1661/1661 [00:55<00:00, 30.09it/s]


28 0.47137693383476953
un estudiante .
a comer pizza .
a mi madre .


100%|██████████████████████████████████████████████████████████████████████████████| 1661/1661 [00:57<00:00, 28.80it/s]


29 0.46212392593276996
un estudiante .
a comer pizza .
a mi madre .
